# RL Agent Performance Evaluation

## 1. Load Data and Models

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load baseline backtest results
baseline_trades = pd.read_csv('data/trades_master.csv')

# Load RL agent backtest results (replace with actual file paths)
ppo_trades = pd.read_csv('data/ppo_trades.csv')
sac_trades = pd.read_csv('data/sac_trades.csv')

## 2. Performance Metrics

In [ ]:
def calculate_performance_metrics(trades, capital=100000):
    pnl = trades['pnl'].sum()
    win_rate = (trades['pnl'] > 0).mean()
    sharpe_ratio = pnl / trades['pnl'].std() if trades['pnl'].std() != 0 else 0
    sortino_ratio = pnl / trades[trades['pnl'] < 0]['pnl'].std() if trades[trades['pnl'] < 0]['pnl'].std() != 0 else 0
    max_drawdown = (trades['pnl'].cumsum().cummax() - trades['pnl'].cumsum()).max()
    return {
        'Total PnL': pnl,
        'Win Rate': win_rate,
        'Sharpe Ratio': sharpe_ratio,
        'Sortino Ratio': sortino_ratio,
        'Max Drawdown': max_drawdown
    }

baseline_metrics = calculate_performance_metrics(baseline_trades)
ppo_metrics = calculate_performance_metrics(ppo_trades)
sac_metrics = calculate_performance_metrics(sac_trades)

metrics_df = pd.DataFrame([baseline_metrics, ppo_metrics, sac_metrics], index=['Baseline', 'PPO', 'SAC'])
print(metrics_df)

## 3. Trade Analysis

In [ ]:
def plot_r_multiple_distribution(trades, title):
    plt.figure(figsize=(10, 6))
    sns.histplot(trades['r_multiple'], bins=50, kde=True)
    plt.title(title)
    plt.xlabel('R-Multiple')
    plt.ylabel('Frequency')
    plt.show()

plot_r_multiple_distribution(baseline_trades, 'Baseline R-Multiple Distribution')
plot_r_multiple_distribution(ppo_trades, 'PPO R-Multiple Distribution')
plot_r_multiple_distribution(sac_trades, 'SAC R-Multiple Distribution')

## 4. Risk Analysis

In [ ]:
def analyze_tail_events(trades, title):
    tail_wins = trades[trades['pnl'] > 0]['pnl'].quantile(0.8)
    tail_losses = trades[trades['pnl'] < 0]['pnl'].quantile(0.2)
    print(f'--- {title} ---')
    print(f'80th Percentile Win: {tail_wins:.2f}')
    print(f'20th Percentile Loss: {tail_losses:.2f}')

analyze_tail_events(baseline_trades, 'Baseline')
analyze_tail_events(ppo_trades, 'PPO')
analyze_tail_events(sac_trades, 'SAC')

## 5. Visualizations

In [ ]:
def plot_equity_curve(trades, title):
    plt.figure(figsize=(10, 6))
    trades['equity_curve'] = trades['pnl'].cumsum()
    plt.plot(trades['equity_curve'])
    plt.title(title)
    plt.xlabel('Trade Number')
    plt.ylabel('Equity')
    plt.show()

plot_equity_curve(baseline_trades, 'Baseline Equity Curve')
plot_equity_curve(ppo_trades, 'PPO Equity Curve')
plot_equity_curve(sac_trades, 'SAC Equity Curve')